### Import libraries 

In [1]:
import os 
import cv2
import sys
import json
from PIL import Image 
from pdf2image import convert_from_path
import pytesseract
from pytesseract import Output

### Converting pdf to jpg images

In [2]:
# Path of the pdf 
PDF_file = "Sanskrit_Text.pdf"
  
# Store all the pages of the PDF in a variable 
pages = convert_from_path(PDF_file, 100) 
  
# Counter to store images of each page of PDF to image 
image_counter = 1
  
# Iterate through all the pages stored above 
for page in pages: 
  
    # Declaring filename for each page of PDF as JPG 
    # For each page, filename will be:  
    # PDF page n -> page_n.jpg 
    filename = "page_"+str(image_counter)+".jpg"
      
    # Save the image of the page in system 
    page.save(filename, 'JPEG') 
  
    # Increment the counter to update filename 
    image_counter = image_counter + 1

### Reading text from images

In [3]:
# Variable to get count of total number of pages 
filelimit = image_counter-1
  
# Iterate from 1 to total number of pages 
for i in range(1, filelimit + 1): 
  
    # Set filename to recognize text from 
    # Again, these files will be:  
    # page_n.jpg 
    filename = "page_"+str(i)+".jpg"
          
    # Variables needed for JSON parsing 
    data0 = {}
    data1 = {}
    count = 0
    
    # Reading the image file
    img = cv2.imread(filename)
    # Reading the image file
    d = pytesseract.image_to_data(img, output_type=Output.DICT, lang='hin')
    n_boxes = len(d['text'])
    
    f = open("JSON_Output.json", "a") 
    
    for i in range(n_boxes):
        if i!=0 and int(d['conf'][i])<0 and d['line_num'][i]!=0:
            #img = cv2.imread(filename)
            count+=1
            data2 = {}
            
            # Assigning the coordinates
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Cropping the boxed statements
            crop_img = img[y:y+h, x:x+w]            
            filepath = filename[:-4]+"extractedImage_"+str(count)+".jpg"
            cv2.imwrite(filepath, crop_img)
            
            # Preparing JSON
            data2['top_left'] = '['+str(x)+','+str(y)+']'
            data2['top_right'] = '['+str(x+w)+','+str(y)+']'
            data2['bottom_left'] = '['+str(x)+','+str(y+h)+']'
            data2['bottom_right'] = '['+str(x+w)+','+str(y+h)+']'
            data1["box"+str(count)] = data2
    data0[filename[:-4]]=data1
    json_data = json.dumps(data0)
    f.write(json_data)
    f.close()
    
    # Displaying image 
    cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('img', 500, 500)
    cv2.imshow('img', img)
    cv2.waitKey(0) 